In [ ]:
import numpy as np
import os
import keras

In [ ]:
!git clone https://github.com/RishiNandha/AWS_Dataset

fatal: destination path 'AWS_Dataset' already exists and is not an empty directory.


In [ ]:
from keras.applications.mobilenet_v2 import MobileNetV2

In [ ]:
from imageio import imread
from skimage.transform import resize
from matplotlib.pyplot import imshow
from keras.applications.mobilenet_v2 import preprocess_input

In [ ]:
objects = {"wood":[1,0,0,0,0,0],"glass":[0,1,0,0,0,0],"metal":[0,0,1,0,0,0],"paper":[0,0,0,1,0,0],"plastic":[0,0,0,0,1,0],"mixed":[0,0,0,0,0,1]}

In [ ]:
total_objects=0
for o in objects:
  for imgs in os.scandir('AWS_Dataset/Dataset/{}'.format(o)):
    total_objects += 1

In [ ]:
data = np.empty((3000,224,224,3))
out = np.empty((3000,6))
vdata = np.empty((600,224,224,3))
vout = np.empty((600,6))

In [ ]:
print(data.shape)

(3000, 224, 224, 3)


In [ ]:

  print("using data set")
  i=0
  n=0
  s=0
  for o in objects:
    n+=1
    print(o)
    print(n)
    for imgs in os.scandir('AWS_Dataset/Dataset/{}'.format(o)):
      im = imread(imgs.path)
      im = preprocess_input(im)
      im = resize(im,output_shape=(224, 224))
      if im.shape == (224,224):
        img_array_expanded = np.expand_dims(im, axis=-1)
        im3 = np.repeat(img_array_expanded, 3, axis=-1)
      else:
        im3 = im

      if(s<n*100):
        vdata[s] = im3[:,:,:3]
        vout[s] = objects[o]
        s+=1
      elif(i<n*500):

        data[i] = im3[:,:,:3]
        out[i] = objects[o]

        i+=1


using data set
wood
1


<ipython-input-9-3f440d763291>:10: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  im = imread(imgs.path)


glass
2
metal
3
paper
4
plastic
5
mixed
6


In [ ]:
im.shape

(224, 224, 3)

In [ ]:
model = MobileNetV2(weights = 'imagenet')

In [ ]:

from tensorflow.keras import layers, Model
from keras.layers import Dense
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam

In [ ]:
aws = Dense(6, activation='softmax')

In [ ]:
aws = aws(model.layers[-2].output)

In [ ]:
from keras import Model
aws_input = model.input
aws_model = Model(inputs = aws_input, outputs=aws)
for layer in aws_model.layers[:-4]:
    layer.trainable = False

In [ ]:
optimizer = Adam(learning_rate=0.009)

In [ ]:
aws_model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
checkpoint = keras.callbacks.ModelCheckpoint('ckpt.keras',monitor = 'val_accuracy', verbose = 1, save_best_only=True, mode ='max')

stop = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 10)
try:
 aws_model.load_weights('ckpt.keras')
except:print('creating new')

In [ ]:


aws_model.fit(x = data, y = out,verbose=2,validation_data=(vdata,vout),batch_size=40,epochs=10,callbacks = [checkpoint,stop])

Epoch 1/5

Epoch 1: val_accuracy improved from -inf to 0.92500, saving model to ckpt.keras
75/75 - 14s - loss: 0.1263 - accuracy: 0.9243 - val_loss: 1.1660 - val_accuracy: 0.9250 - 14s/epoch - 191ms/step
Epoch 2/5

Epoch 2: val_accuracy did not improve from 0.92500
75/75 - 4s - loss: 0.0923 - accuracy: 0.9277 - val_loss: 1.8108 - val_accuracy: 0.8933 - 4s/epoch - 60ms/step
Epoch 3/5

Epoch 3: val_accuracy improved from 0.92500 to 0.93333, saving model to ckpt.keras
75/75 - 6s - loss: 0.1010 - accuracy: 0.9330 - val_loss: 1.0665 - val_accuracy: 0.9333 - 6s/epoch - 75ms/step
Epoch 4/5

Epoch 4: val_accuracy improved from 0.93333 to 0.93667, saving model to ckpt.keras
75/75 - 5s - loss: 0.1349 - accuracy: 0.9280 - val_loss: 1.0144 - val_accuracy: 0.9367 - 5s/epoch - 66ms/step
Epoch 5/5

Epoch 5: val_accuracy did not improve from 0.93667
75/75 - 5s - loss: 0.1093 - accuracy: 0.9330 - val_loss: 1.0034 - val_accuracy: 0.9317 - 5s/epoch - 67ms/step


In [ ]:
del data
del out
import gc
gc.collect()
predictions=aws_model.predict(vdata)

In [ ]:
for t in range(0,600):
  predictions[t]=Ob[np.argmax(predictions[t])]
from sklearn.metrics import f1_score
print(f1_score(vout,predictions,average=None))

In [ ]:
keras.backend.set_value(aws_model.optimizer.learning_rate, 0.005)
aws_model.fit(x = data, y = out ,verbose=2,validation_data=(vdata,vout),initial_epoch=5,epochs=15,callbacks = [checkpoint,stop])

In [ ]:
Ob=[[1,0,0,0,0,0] ,[0,1,0,0,0,0],[0,0,1,0,0,0],[0,0,0,1,0,0],[0,0,0,0,1,0],[0,0,0,0,0,1]]

In [ ]:
predictions

In [ ]:

mismatch = np.zeros(6,)
l=0
pos = 0;
for prediction in predictions:
  if np.argmax(prediction) != np.argmax(vout[l]):
    mismatch[np.argmax(vout[l])] += 1
    print(predictions[l]);
  l+=1

In [ ]:
aws_model.save('AWS')

In [ ]:
aws_model.summary()

In [ ]:
mismatch
vout[301]